In [9]:
import os
import asyncio
from openai import AsyncAzureOpenAI
from openai import OpenAIError
from agents import Agent, Runner, OpenAIChatCompletionsModel, set_tracing_disabled

from azure.identity import DefaultAzureCredential, get_bearer_token_provider

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)


In [11]:
from agents import Agent, ModelSettings, function_tool

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="o3-mini",
    tools=[get_weather],
)

In [12]:
from dataclasses import dataclass


@dataclass
class UserContext:
  uid: str
  is_pro_user: bool

  async def fetch_purchases() -> list[str]:
     return ...

agent = Agent[UserContext](
    ...,
)

In [13]:
from pydantic import BaseModel
from agents import Agent


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

agent = Agent(
    name="Calendar extractor",
    instructions="Extract calendar events from text",
    output_type=CalendarEvent,
)

In [ ]:

# Disable tracing since we're using Azure OpenAI
set_tracing_disabled(disabled=True)

deployment = "gpt-4o"

async def run_agent():
    try:
        # Create the Async Azure OpenAI client
        client = AsyncAzureOpenAI(
            api_version="2023-09-01-preview",
            azure_endpoint="https://aihub6750316290.openai.azure.com/",
            azure_ad_token_provider=token_provider,  # Use DefaultAzureCredential for authentication

        )

        # Configure the agent with Azure OpenAI
        agent = Agent(
            name="Assistant",
            instructions="You are a helpful assistant",
            model=OpenAIChatCompletionsModel(
                model=deployment,
                openai_client=client,
            )
        )

        result = await Runner.run(agent, "Write a haiku about recursion in programming.")
        print(result.final_output)

    except OpenAIError as e:
        print(f"OpenAI API Error: {str(e)}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

await run_agent()

Code calls itself, deep—  
Patterns loop, yet grow anew.  
Logic finds its end.


In [ ]:
from agents import Agent

booking_agent = Agent(...)
refund_agent = Agent(...)

triage_agent = Agent(
    name="Triage agent",
    instructions=(
        "Help the user with their questions."
        "If they ask about booking, handoff to the booking agent."
        "If they ask about refunds, handoff to the refund agent."
    ),
    handoffs=[booking_agent, refund_agent],
)